# Dataset Cultura Cearense
**Autor:** Danne Makleyston Gomes Pereira <br>
**Data da criação:** 24 jun 2021 <br>
**Última atualização:** -- <br>

### Descrição do dataset
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

### Formato do dataset
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.



## Desenvolvimento do *dataset* cultura_cearence
### Instalação das libs necessárias

In [ ]:
# Instalando o 'reuests' para consultar páginas web
# Instalando o 'beautifulsoup4' para fazer o web scrawler
!pip install requests beautifulsoup4

In [ ]:
# Instalando o 'tabula-py' para ler tabelas em PDFs
!pip install tabula-py

In [ ]:
# Instalando o 'tika' para ler textos em PDFs
!pip install tika

### Importação das libs necessárias para a construção do dataset

In [424]:
# Importação da lib para consultar uma URL
import requests

# Importação da lib para realizar o web scrawler
from bs4 import BeautifulSoup

# Importação da lib para tratar expressões regulares (regex)
import re

# Importação da lib para ler tabelas em PDFs
import tabula

# Importação da lib para tratar DataFrames e Series
import pandas as pd

# Importação da lib para let textos em PDFs
from tika import parser 

# Importação da lib para manipular de arrays, sobre tudo de números
import numpy as np

# Importação da lib para manipular data
from datetime import datetime

# Importação da lib para conversão de str para decimal
from decimal import Decimal

### Etapa 01: Coletando todas as URLs das páginas dos editais publicados

In [2]:
# Página inicial da seção de editais
url_home_page = 'http://editais.cultura.ce.gov.br/'

# Lista de URL de páginas dos editais publicados
url_notices = []

# HTML da página inicial
html_home_page = requests.get(url_home_page).text

# Convertendo para um objeto soup
soup_home_page = BeautifulSoup(html_home_page, 'html5lib')

# Capturando todas as tags HTML 'a'
tags_a = soup_home_page.select('div.et_pb_section_2 div.et_pb_row div.et_pb_css_mix_blend_mode_passthrough div.et_pb_module a')

# Coletando todos os links presentes no atributo 'href' da tag 'a' e inserindo na lista 'url_notices'
for a in tags_a:
    link = a['href'].strip()
    if not 'https' in link: # Foi observado que todos os links são apenas 'http'
        url_notices.append(link)

In [6]:
# Apresentando as URLs dos editais encontrados
pd.DataFrame(url_notices, columns = ['url_notices']).head(10)

,url_notices
0,http://editais.cultura.ce.gov.br/2021/04/06/9909/
1,http://editais.cultura.ce.gov.br/2020/03/05/ed...
2,http://editais.cultura.ce.gov.br/2019/09/11/xi...
3,http://editais.cultura.ce.gov.br/2018/09/28/ch...
4,http://editais.cultura.ce.gov.br/2021/01/29/ed...
5,http://editais.cultura.ce.gov.br/2019/10/16/xi...
6,http://editais.cultura.ce.gov.br/2021/03/15/fe...
7,http://editais.cultura.ce.gov.br/2021/02/03/xi...
8,http://editais.cultura.ce.gov.br/2019/04/17/ed...
9,http://editais.cultura.ce.gov.br/2020/10/07/ch...


### Etapa 2: Desenvolvimento de funções para a coleta dos dados

#### Coleta do título do edital:

In [7]:
def getTitle(soup):
    # Localiza o título na página do edital
    for t in soup.select('h1.entry-title'): #.et_pb_title_container 
        
        # Coleta apenas o texto entre as tags
        title = t.get_text()
        
        # Retorna todos os resultados com letras maiúsculas
        return title.upper()

#### Coleta da data de inscrição:

In [8]:
def getRegistrationDateOld(soup):
    
    # Especificação de REGEX para a coleta de datas
    patterns = ['([0-9]+ ([a-zA-Z]+( [a-zA-Z]+)+) [0-9]+ ([a-zA-Z]+( [a-zA-Z]+)+) [0-9]+)',
                '(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]|(?:Jan|Mar|May|Jul|Aug|Oct|Dec)))\1|(?:(?:29|30)(\/|-|\.)(?:0?[1,3-9]|1[0-2]|(?:Jan|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))\2))(?:(?:1[6-9]|[2-9]\d)?\d{2})$|^(?:29(\/|-|\.)(?:0?2|(?:Feb))\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|^(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9]|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep))|(?:1[0-2]|(?:Oct|Nov|Dec)))\4(?:(?:1[6-9]|[2-9]\d)?\d{2})',
                '(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012]).*?(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](18|19|20)\d\d',
                '([0-9]+(/[0-9]+)+)+.*?([0-9]+(/[0-9]+)+)',
                '[0-9](\s([a-zA-Z]+\s)+)[0-9]+(\s([a-zA-Z]+\s)+)[0-9]+',
                '[0-9].*([a-zA-Z]+( [a-zA-Z]+)+).*[0-9].*([a-zA-Z]+( [a-zA-Z]+)+).*(\s([a-zA-Z]+\s)+)[0-9]+'
                ]
    
    # Localiza as informações sobre 'Inscrições' na página do edital
    for li in soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description p'):
        
        # Coleta apenas o texto entre as tags
        line = li.get_text()
        
        # Identifica se a seção trata de assuntos relacionados às inscrições
        index = line.strip().find('Inscriç')
        end = index + 150
        
        if index >= 0:
            # Cada padrão REGEX é analisado na página do edital
            for pattern in patterns:
                m = re.search(pattern, line[index:end], flags=re.IGNORECASE)
                if m:
                    return m.group()
    return ''

In [12]:
def getRegistrationDate(soup):
    
    patterns = ['([0-9]+ ([a-zA-Z]+( [a-zA-Z]+)+) [0-9]+ ([a-zA-Z]+( [a-zA-Z]+)+) [0-9]+)',
            '(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]|(?:Jan|Mar|May|Jul|Aug|Oct|Dec)))\1|(?:(?:29|30)(\/|-|\.)(?:0?[1,3-9]|1[0-2]|(?:Jan|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))\2))(?:(?:1[6-9]|[2-9]\d)?\d{2})$|^(?:29(\/|-|\.)(?:0?2|(?:Feb))\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|^(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9]|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep))|(?:1[0-2]|(?:Oct|Nov|Dec)))\4(?:(?:1[6-9]|[2-9]\d)?\d{2})',
            '(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012]).*?(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](18|19|20)\d\d',
            '([0-9]+(/[0-9]+)+)+.*?([0-9]+(/[0-9]+)+)',
            '([0-9]+(/[0-9]+)+).*a.*([0-9]+(/[0-9]+)+)',
            '[0-9](\s([a-zA-Z]+\s)+)[0-9]+(\s([a-zA-Z]+\s)+)[0-9]+',
            '[0-9].*([a-zA-Z]+( [a-zA-Z]+)+).*[0-9].*([a-zA-Z]+( [a-zA-Z]+)+).*(\s([a-zA-Z]+\s)+)[0-9]+'
            ]
    
    # Localiza as informações sobre 'Inscrições' na página do edital
    if len(soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description p')) > 0:
        l = soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description')
    elif(len(soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description ul li')) > 0):
        l = soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description ul li')
    else:
        l = soup.select('div.et_pb_blurb_0 div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description p')

    for li in l:
        index = -1
        
        # Coleta apenas o texto entre as tags
        line = li.get_text()
        
        # Verifica se há questões relacionadas às inscrições nessa linha da seção
        res = re.search('inscri', line, re.IGNORECASE)
        
        if res:
            # Encontra a primeira instância do padrão encontrado
            index = line.strip().find(res.group())
            # Delimita uma substrig para trabalhar na busca pelas datas (150 é um valor arbritrário definido com base na observação dos dados)
            end = index + 150
            
        if index >= 0:
            # Verifica a string com cada padrão REGEX
            for pattern in patterns:
                m = re.search(pattern, line[index:end], flags=re.IGNORECASE)
                if m:
                    return m.group()
    return ''

#### Coleta da quantidade de prorrogações:

In [27]:
def getAmountDateExtensions(soup):
    
    # Definição do padrão REGEX
    pattern = 'Prorroga.*'
    
    sum_amount = 0
    
    # Localiza as informações na página web do edital
    for p in soup.select('div.et_pb_blurb_content div.et_pb_blurb_container div.et_pb_blurb_description p'):
        
        # Coleta apenas o texto entre as tags
        line = p.get_text()
        
        # Atribui à somatória
        amount = len(re.findall(pattern, line, flags=re.IGNORECASE))
        sum_amount = sum_amount + amount
        
    return str(sum_amount)

#### Coleta da URL do arquivo PDF do edital:

In [22]:
def getPDFNoticeURL(soup):
    
    # Localiza os dados na página web do edital
    for sec in soup.select('div.et_pb_blurb_container .et_pb_module_header'):
        
        # Verifica se a seção trata de dados sobre as publicações do edital
        if sec.get_text().find('Publicações') >= 0:
            
            # Localiza os dados das publicações do edital
            for a in soup.select('div.et_pb_blurb_container .et_pb_blurb_description a'):
                
                # Coletando apenas os links presentes no atributo 'href'
                link = a['href'].strip()
                
                # Selecionando apenas as URLs que direcionam para um arquivo PDF
                if link.find('.pdf') >= 0:
                    if link[0:2] == '//': # Tratamento para caso esteja faltando o 'http' na frente do link
                        link = 'http:'+link
                    return link
    return ''

#### Coleta das categorias para o qual o edital se destina:

In [11]:
def getCategories(soup):
    
    # Localiza os dados na página web do edital
    for sec in soup.select('div.et_pb_blurb_container'):
        
        # Coleta o título da seção
        for section in sec.select('.et_pb_module_header'):
            section_label = section.get_text()
            
            # Verifica se o título da seção é 'Linguagens', 'Categorias' ou 'Segmentos' (termos usados pelo orgão)
            if section_label.find('Linguagens') >= 0 or section_label.find('Categorias') >= 0 or section_label.find('Segmentos') >= 0:
                
                # Localiza os elementos 'p' dentro da seção de categorias
                for value in sec.select('.et_pb_blurb_description p'):
                    return value.get_text().strip()
    return ''

#### Coleta da quantidade de inscritos:

In [761]:
def getAmoutRegisters(df_registred_projects):
    amount = 0
    for page in range(len(df_registred_projects)):
        for i in range(len(df_registred_projects[page])):
            if df_registred_projects[page].iloc[i][0] is not np.nan:
                if isinstance(df_registred_projects[page].iloc[i][0], (int, np.integer)):
                    amount = df_registred_projects[page].iloc[i][0]
                else:
                    pass    
    return amount

#### Coleta dos projetos aprovados:

In [771]:
def getApprovedProjects(df_approved_projects):
    approved_projects_list = []
    for page in range(len(df_approved_projects)):
        for i in range(len(df_approved_projects[page])):
            if df_approved_projects[page].iloc[i][6] is not np.nan:
                check_approval = re.search(r'CLASSIFICAD.', df_approved_projects[page].iloc[i][6], flags=re.IGNORECASE)
                if check_approval:
                    approved_projects_list.append([df_approved_projects[page].iloc[i][2], df_approved_projects[page].iloc[i][3], df_approved_projects[page].iloc[i][4]])
    return approved_projects_list

#### Coleta do valor orçamentário disponível para o edital:

In [19]:
def getValue(soup):
    
    # Definição do padrão para valores monetários do Brasil (BRL)
    pattern = '\s*(?:[1-9]\d{0,2}(?:\.\d{3})*|0),\d{2}'
    
    # Localiza os dados na página web do edital
    for res in soup.select('.et_pb_blurb_content .et_pb_blurb_container .et_pb_module_header'):
        
        # Coleta apenas o texto entre as tags
        line = res.get_text()
        
        # Verifica se a seção de dados trata assuntos relacionados ao valor do edital
        if line.find('Valor d') >= 0:
            
            # Localiza a seção de dados que possui os valores monetários
            for res2 in soup.select('.et_pb_blurb_content .et_pb_blurb_container div.et_pb_blurb_description'):
                
                # Coleta apenas o texto entre as aspas
                value = res2.get_text().strip()
                
                # Verifica se o padrão REGEX é atendido nesse trecho de texto
                m = re.search(pattern, value, flags=re.IGNORECASE)
                if m is not None:
                    return m.group()
    return ''

In [66]:
def getValuePDF(url_page):

    try:
        # Converte o PDF em arquivo de texto utilizando o 'tika'
        raw = parser.from_file(url_page)
        
        # Define o padrão REGEX para valores monetário no Brasil (BRL)
        pattern = '\s*(?:[1-9]\d{0,2}(?:\.\d{3})*|0),\d{2}'
        
        # Pesquisa pelo padrão REGEX dentro do texto do PDF
        res = re.search(pattern, raw['content'])
        
        if res:
            print(res)
            value = res.group(0)
            return value
    except:
        return ''

### Geração do Dataset Cultura Cearense

#### Definição do dataset:

In [67]:
# Definição do dataset
dataset = []

#### Gerando o dataset com os valores coletados:

In [68]:
# Variáveis temporária
dataset2 = []
url_notices2 = ['http://editais.cultura.ce.gov.br/2021/04/06/fomentoparagruposdosciclosdaculturatradicionaldoce/']


# Navega entre todos as URL das páginas dos editais
for url in url_notices:
    
    # Coleta o HTML da página web do edital
    html_page = requests.get(url).text
    
    # Converte o HTML da página para um objeto SOUP
    soup = BeautifulSoup(html_page, 'html5lib')
    
    # Coleta o título do edital
    _title = getTitle(soup)
    
    # Coleta a data de inscrição
    _registration_date = getRegistrationDate(soup)
    
    # Coleta a quantidade de prorrogações houve    
    _amount_date_extensions = getAmountDateExtensions(soup)
    
    # Coleta a quantidade de iscrições
    _amount_registers = getAmoutRegisters(soup)
    
    # Coleta os projetos aprovados
    _aproved_projects = getApprovedProjects(soup)
    
    # Coleta das categorias do edital
    _categories = getCategories(soup)
    
    # Coleta da cidade do projeto aprovado
    _city = getCity(soup)
    
    # Coleta da URL pdf do edtial
    _url_pdf_notice = getPDFNoticeURL(soup)   
    
    # Coleta do valor orçamentário do edital
    _value = getValue(soup)
    
    # Se a informação não estiver na página web do edital, então busca-se no arquivo PDF do edital
    if _value == '' and not _url_pdf_notice == '':
        _value = getValuePDF(_url_pdf_notice)
        
    try:
        # Inserindo os valores coletados no dataset
        dataset.append([
            _title, _registration_date, _amount_date_extensions, 
            _amount_registers, _aproved_projects, _categories, _city,
            _value, url, _url_pdf_notice
        ])
    except:
        print('')

2021-06-26 13:41:21,041 [MainThread  ] [INFO ]  Retrieving https://mapacultural.secult.ce.gov.br/files/opportunity/3154/edital-2021-fomento.pdf to /tmp/files-opportunity-3154-edital-2021-fomento.pdf.


<re.Match object; span=(16932, 16945), match=' 3.320.000,00'>


2021-06-26 13:41:25,557 [MainThread  ] [INFO ]  Retrieving http://arteurgente.com.br/editais/chamada-publica-n-01-arte-urgente-janelas-formativas.pdf to /tmp/editais-chamada-publica-n-01-arte-urgente-janelas-formativas.pdf.


<re.Match object; span=(4498, 4509), match=' 192.000,00'>


2021-06-26 13:41:29,775 [MainThread  ] [INFO ]  Retrieving https://mapacultural.secult.ce.gov.br/files/opportunity/1544/xiv_edital_-_ceara_de_cinema_e_video.pdf to /tmp/files-opportunity-1544-xiv_edital_-_ceara_de_cinema_e_video.pdf.


<re.Match object; span=(15865, 15878), match=' 8.200.000,00'>


2021-06-26 13:41:33,277 [MainThread  ] [INFO ]  Retrieving https://www.secult.ce.gov.br/wp-content/uploads/sites/43/2021/03/CONVOCATORIA-FESTA-DO-SOL-1.pdf to /tmp/wp-content-uploads-sites-43-2021-03-convocatoria-festa-do-sol-1.pdf.


<re.Match object; span=(15877, 15886), match=' 4.000,00'>


2021-06-26 13:41:44,087 [MainThread  ] [INFO ]  Retrieving http://mapa.cultura.ce.gov.br/files/project/859/minuta-praverbanda-convocatoria.pdf to /tmp/files-project-859-minuta-praverbanda-convocatoria.pdf.
2021-06-26 13:41:45,252 [MainThread  ] [INFO ]  Retrieving http://mapa.cultura.ce.gov.br/files/opportunity/1285/edital-tesouros-vivos-da-cultura-2018.pdf to /tmp/files-opportunity-1285-edital-tesouros-vivos-da-cultura-2018.pdf.
2021-06-26 13:41:47,597 [MainThread  ] [INFO ]  Retrieving http://rede.cultura.ce.gov.br/editais/wp-content/uploads/sites/18/2017/10/edital-de-convocao-eleio-cepc-ce-20182f2020.pdf to /tmp/editais-wp-content-uploads-sites-18-2017-10-edital-de-convocao-eleio-cepc-ce-20182f2020.pdf.
2021-06-26 13:41:51,217 [MainThread  ] [INFO ]  Retrieving http://mapa.cultura.ce.gov.br/files/opportunity/1323/edital_programa_formacao_artes_cenicas.pdf to /tmp/files-opportunity-1323-edital_programa_formacao_artes_cenicas.pdf.
2021-06-26 13:41:57,414 [MainThread  ] [INFO ]  Retrie

#### Formação do DataFrame (Pandas):

In [625]:
# Definindo os títulos das colunas
cols = ['title', 'registration_date', 'amount_date_extensions', 
            'amount_registers', 'aproved_projects', 'categories', 'city',
            'value', 'url', 'url_pdf_notice']

# Gerando um Pandas DataFrame
df = pd.DataFrame(data = dataset, columns = cols)

### Manipulações no DataFrame

In [433]:
# Backup do DataFrame
bkp_df = df
# df = bkp_df

In [166]:
# Mostrando o df
df

,title,registration_date,amount_date_extensions,amount_registers,aproved_projects,categories,city,value,url,url_pdf_notice
0,EDITAL DE FOMENTO PARA GRUPOS DOS CICLOS DA CULTURA TRADICIONAL POPULAR DO CEARÁ,06 de abril a 03 de maio 2021,2,,,,,"3.320.000,00",http://editais.cultura.ce.gov.br/2021/04/06/9909/,https://mapacultural.secult.ce.gov.br/files/opportunity/3154/edital-2021-fomento.pdf
1,EDITAL CULTURA INFÂNCIA 2020,05/04/2020 a 27/04/2020,1,,,,,"1.500.000,00",http://editais.cultura.ce.gov.br/2020/03/05/edital-cultura-infancia-2020/,https://mapacultural.secult.ce.gov.br/files/opportunity/1642/resultado_preliminar_habilita%C3%A7%C3%A3o_da_inscri%C3%A7%C3%A3o.pdf
2,XII EDITAL CEARÁ DE INCENTIVO ÀS ARTES,11/11 a 06/12/2019,1,,,I – Artes Visuais; II – Dança; III – Circo; IV – Humor&Comédia; V – Fotografia; VI – Literatura; VII – Música; VIII – Prêmio Alberto Nepomuceno de Composição Musical; IX – Prêmio Pedro Boca Rica de Teatro de Bonecos; X – Teatro,,"6.877.492,00",http://editais.cultura.ce.gov.br/2019/09/11/xii-edital-ceara-de-incentivo-as-artes/,https://mapacultural.secult.ce.gov.br/files/opportunity/1566/avalia%C3%87%C3%83o_e_sele%C3%87%C3%83o_da_proposta_-_result_prelim._literatura_(2).pdf
3,EDITAL DE CHAMADA PÚBLICA DE CREDENCIAMENTO DE PARECERISTA DA SECRETARIA DA CULTURA DO ESTADO DO CEARÁ,10/18. Inscrições | 24/10,1,,,"1. Artes Visuais2. Audiovisual, Arte Digital e Novas Mídias3. Circo4. Dança5. Fotografia6. Humor7. Literatura8. Música9. Teatro10. Patrimônio Cultural Material11. Patrimônio Cultural Imaterial12. Museus e Memória13. Moda e Design14 .Intersetorialidades das Políticas Públicas de Cultura, Gestão e produção Cultural",,"3.234,00",http://editais.cultura.ce.gov.br/2018/09/28/chamada-publica-de-credenciamento-de-pareceristas/,http://mapa.cultura.ce.gov.br/files/opportunity/1094/edital-de-chamada-publica-de-credenciamento-de-pareceristas-para-selecao-e-avaliacao-de-projetos-do-edital-de-incentivo-as-artes.pdf
4,EDITAIS – ARTE URGENTE,1 a 12 de fevereiro de 2021,2,,,,,"192.000,00",http://editais.cultura.ce.gov.br/2021/01/29/editais-arte-urgente/,http://arteurgente.com.br/editais/chamada-publica-n-01-arte-urgente-janelas-formativas.pdf
5,XIV EDITAL CEARÁ CINEMA E VÍDEO,18/11/2019 a 17/12/2019,2,,,Categoria Produção,,"8.200.000,00",http://editais.cultura.ce.gov.br/2019/10/16/xiv-edital-ceara-cinema-e-video/,https://mapacultural.secult.ce.gov.br/files/opportunity/1544/xiv_edital_-_ceara_de_cinema_e_video.pdf
6,FESTA DO SOL – CONVOCATÓRIA,15/03/2021 a 25/03/2021,1,,,,,"4.000,00",http://editais.cultura.ce.gov.br/2021/03/15/festa-do-sol-convocatoria/,https://www.secult.ce.gov.br/wp-content/uploads/sites/43/2021/03/CONVOCATORIA-FESTA-DO-SOL-1.pdf
7,XIII EDITAL MECENAS DO CEARÁ,3 de fevereiro a 9 de março de 2021,3,,,,,"26.000.000,00",http://editais.cultura.ce.gov.br/2021/02/03/xiii-edital-mecenas-do-ceara/,https://mapacultural.secult.ce.gov.br/files/opportunity/2789/1%C2%BA_aditivo_xiii_edital_mecenas_(1)-1.pdf
8,EDITAL DE CHAMAMENTO PÚBLICO PROJETO CIRCULA CEARÁ,17/4/19 a 17/5/19,2,,,CircoOficina Acessibilidade AtitudinalCultura Popular TradicionalDançaFotografiaIntervenção UrbanaLiteraturaMúsicaTeatro,,"613.800,00",http://editais.cultura.ce.gov.br/2019/04/17/edital-de-chamamento-publico-projeto-circula-ceara,http://mapacultural.secult.ce.gov.br/files/opportunity/1385/resultados_circula_-_literatura_resultado_ap%C3%93s_recurso.pdf
9,CHAMADA PÚBLICA PARA SELEÇÃO E AQUISIÇÃO DE ACERVO BIBLIOGRÁFICO DE PRODUÇÃO CEARENSE PARA O SISTEMA ESTADUAL DE BIBLIOTECAS PÚBLICAS DO CEARÁ,07/10/2020 a 13/10/2020,1,,,,,"2.000.000,00",http://editais.cultura.ce.gov.br/2020/10/07/chamada-publica-para-selecao-e-aquisicao-de-acervo-bibliografico-de-producao-cearense-para-o-sistema-estadual-de-bibliotecas-publicas-do-ceara/,https://mapacultural.secult.ce.gov.br/files/opportunity/2432/edital_aquisi%C3%A7%C3%A3o_de_acervo_bibliogr%C3%A1fico__leiab_2020_10_13_-_comobserva%C3%A7%C3%B5esealtera%C3%A7%C3%B5esdaasjur_aditivo.pdf


#### Tratando o campo 'Categorias'
O objetivo é tornar os dados da seguinte forma: 'cat1; cat2; cat3'

In [626]:
df[df['categories'].notna()][['categories']].head()

,categories
0,
1,
2,I – Artes Visuais; II – Dança; III – Circo; IV – Humor&Comédia; V – Fotografia; VI – Literatura; VII – Música; VIII – Prêmio Alberto Nepomuceno de Composição Musical; IX – Prêmio Pedro Boca Rica de Teatro de Bonecos; X – Teatro
3,"1. Artes Visuais2. Audiovisual, Arte Digital e Novas Mídias3. Circo4. Dança5. Fotografia6. Humor7. Literatura8. Música9. Teatro10. Patrimônio Cultural Material11. Patrimônio Cultural Imaterial12. Museus e Memória13. Moda e Design14 .Intersetorialidades das Políticas Públicas de Cultura, Gestão e produção Cultural"
4,


In [627]:
# Padrões REGEX para tratamento das categorias
category_patterns = [
            [r'', np.nan], # Altera as linhas vazias para NaN
            [r'((?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})+\s+–\s)', ';'], # Números romanos
            [r'((?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})+\s+)', ';'],
            [r'([0-9]{1,2})+(\s+)?\.', '; '], # Retirando numeração de ordem
            [r';\s+;', ';'], # Espaços em branco
            [r'^;', ''], # Retirando ';' do começo
            [r';', '; '], # Atribuindo um espaço após o ';'
            [r'\s+;', ';'], # Retirando espaços antes do ';'
            [r'(c|C)ategoria;', ''],
            [r'(\s+);', ';']
           ] 

for pattern in category_patterns:
    df['categories'].replace(pattern[0], pattern[1], inplace=True, regex = True)
    
df['categories'] = df['categories'].str.upper()

In [479]:
# **** Obsoleto ****
# Split palavras
def splitWordCategories(item):
    # Padrão do regex
    pattern = r'([A-Z][a-záàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ ]+)?'

    res = re.findall(pattern, item)
    
    item_formated = ''
    
    for w in res:
        item_formated = item_formated + '; ' + w

    return item_formated

In [474]:
# **** Obsoleto ****
# for i in range(len(df)):
#     if df.iloc[i]['categories'] and df.iloc[i]['categories'] is not np.nan:
#         item = splitWordCategories(df.iloc[i]['categories'])
#         if item:
#             df.iloc[i]['categories'] = item

In [628]:
# Selecionando apenas as linhas com valores                         
df_categories_notna = df[df['categories'].notna()]

df_categories_notna[['categories']]

,categories
2,ARTES VISUAIS; DANÇA; CIRCO; HUMOR&COMÉDIA; FOTOGRAFIA; LITERATURA; MÚSICA; PRÊMIO ALBERTO NEPOMUCENO DE COMPOSIÇÃO MUSICAL; PRÊMIO PEDRO BOCA RICA DE TEATRO DE BONECOS; TEATRO
3,"ARTES VISUAIS; AUDIOVISUAL, ARTE DIGITAL E NOVAS MÍDIAS; CIRCO; DANÇA; FOTOGRAFIA; HUMOR; LITERATURA; MÚSICA; TEATRO; PATRIMÔNIO CULTURAL MATERIAL; PATRIMÔNIO CULTURAL IMATERIAL; MUSEUS E MEMÓRIA; MODA E DESIGN; INTERSETORIALIDADES DAS POLÍTICAS PÚBLICAS DE CULTURA, GESTÃO E PRODUÇÃO CULTURAL"
5,CATEGORIA PRODUÇÃO
8,CIRCOOFICINA ACESSIBILIDADE ATITUDINALCULTURA POPULAR TRADICIONALDANÇAFOTOGRAFIAINTERVENÇÃO URBANALITERATURAMÚSICATEATRO
18,"GRUPOS DE TRADIÇÃO E PROJEÇÃO (CARNAVAL, PAIXÃO, JUNINO, NATAL) E OUTRAS EXPRESSÕES DA CULTURA TRADICIONAL POPULAR PRESERVAÇÃO DO PATRIMÔNIO CULTURAL CEARENSE"
24,GRUPOS DE TRADIÇÃO NATALINA; GRUPOS DE PROJEÇÃO (PARAFOLCLÓRICOS) DO CICLO NATALINO; PRESÉPIOS; CORAL; MOSTRAS REGIONAIS NATALINAS;; MOSTRA ESTADUAL CEARÁ NATAL DE LUZ
33,QUADRILHAS JUNINAS ADULTAS QUADRILHAS JUNINAS INFANTIS QUADRILHAS JUNINAS DA DIVERSIDADE FESTIVAL REGIONAL DE QUADRILHAS JUNINAS
35,BAILES E MATINÊS
36,DOAÇÃO PATROCÍNIO INVESTIMENTO
38,DOAÇÃO PATROCÍNIO INVESTIMENTO


#### Tratando o campo data de inscrição
O objetivo é transformar os formatos para a seguinte estrutura de dada:
* As datas deverão segui a ordem `dd/mm/aaaa`.
* A coluna **registration_date** deverá se transformar em duas colunas **registration_date_init** e **registrantion_date_end**, no qual o primeiro comportará a data inicial das inscrições e o segundo a data de término das inscrições.

In [630]:
df[df['registration_date'].notna()][['registration_date']]

,registration_date
0,06 de abril a 03 de maio 2021
1,05/04/2020 a 27/04/2020
2,11/11 a 06/12/2019
3,10/18. Inscrições | 24/10
4,1 a 12 de fevereiro de 2021
5,18/11/2019 a 17/12/2019
6,15/03/2021 a 25/03/2021
7,3 de fevereiro a 9 de março de 2021
8,17/4/19 a 17/5/19
9,07/10/2020 a 13/10/2020


In [631]:
# Aplicando várias camadas de REGEX
date_patterns = [
                    ['', np.nan, True], # Altera as linhas vazias para NaN
                    [r'/17\b', '/2017', True], # Tornando os anos 'aaaa'
                    [r'/18\b', '/2018', True], # Tornando os anos 'aaaa'
                    [r'/19\b', '/2019', True], # Tornando os anos 'aaaa'
                    [r'/20\b', '/2020', True], # Tornando os anos 'aaaa'
                    [r'/21\b', '/2021', True], # Tornando os anos 'aaaa'
                    ['janeiro',   '/01/', True],
                    ['fevereiro', '/02/', True],
                    ['março',     '/03/', True],
                    ['abril',     '/04/', True],
                    ['maio',      '/05/', True],
                    ['junho',     '/06/', True],
                    ['julho',     '/07/', True],
                    ['agosto',    '/08/', True],
                    ['setembro',  '/09/', True],
                    ['outubro', '  /10/', True],
                    ['novembro',  '/11/', True],
                    ['dezembro',  '/12/', True],
                    [r'\bde\b',   '/', True],
                    [r'\//', '/', True],
                    [r'\/ /', '/', True],
                    [r'\s/', '/', True],
                    [r'/\s', '/', True],
                    ['\.', '/', True],
                ] 

for pattern in date_patterns:
    df['registration_date'].replace(pattern[0], pattern[1], inplace=True, regex = pattern[2])
    
df_registration_date_notna = df[df['registration_date'].notna()]
df_registration_date_notna[['registration_date']]

,registration_date
0,06/04/a 03/05/2021
1,05/04/2020 a 27/04/2020
2,11/11 a 06/12/2019
3,10/2018/ Inscrições | 24/10
4,1 a 12/02/2021
5,18/11/2019 a 17/12/2019
6,15/03/2021 a 25/03/2021
7,3/02/a 9/03/2021
8,17/4/2019 a 17/5/2019
9,07/10/2020 a 13/10/2020


In [632]:
# Split registraion_date
df[['registration_date_init', 'registration_date_end']] = df['registration_date'].str.split('a ', 1, expand=True)
df[['registration_date_init', 'registration_date_end']]

,registration_date_init,registration_date_end
0,06/04/,03/05/2021
1,05/04/2020,27/04/2020
2,11/11,06/12/2019
3,10/2018/ Inscrições | 24/10,None
4,1,12/02/2021
5,18/11/2019,17/12/2019
6,15/03/2021,25/03/2021
7,3/02/,9/03/2021
8,17/4/2019,17/5/2019
9,07/10/2020,13/10/2020


In [633]:
# Retorna o ano de uma data
def getYear(date):
    if date and (date is not np.nan):
        res = re.search('/\d{4}$', date)
        if res:
            return res.group()

In [634]:
# Retorna o mês de uma data
def getMonth(date):
    if date and (date is not np.nan):
        res = re.search('/\d{1,2}/', date)
        if res:
            return res.group()

In [635]:
# Selecionando apenas as datas no formato desejado
def getDateFormated(date):
    
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4})' # identifica o formato desejado
    
    new_date_match = re.search(pattern, date)
    
    if new_date_match:
        new_date_string = new_date_match.group()
        new_date_dt = datetime.strptime(new_date_string, '%d/%m/%Y')
        new_date_string = new_date_dt.strftime('%d/%m/%Y')
        return new_date_string
    else:
        return np.nan
    

In [636]:
def fixDateInit(date_init, date_end):
        
    patterns_date_ = [
                    r'^(\s+)?(^/)?\b\d{1,2}(\s+)?$', # identifica que há apenas o dia e soma com o mês
                    r'\b([0-9]{1,2})/([0-9]{1,2})((\s+|/)?)(\s+)?$', # identifica que possui o dia e o mês e soma com ano
                    r'/(\s+)?/', # identifica duas barras
                    r'\s+?' # identifica espaços em branco
                ]
    
    month_miss = re.search(patterns_date_[0], date_init) 

    if month_miss:
        month = getMonth(date_end)
        if month:
            date_init = date_init + month
    
    date_init = re.sub(patterns_date_[3], '', date_init)

    year_miss = re.search(patterns_date_[1], date_init)
    
    if year_miss:
        year = getYear(date_end)
        if year:
            date_init = date_init + year
    
    date_init = re.sub(patterns_date_[2], '/', date_init)
    date_init = re.sub(patterns_date_[3], '', date_init)
    
    return getDateFormated(date_init)
    

In [637]:
# Corrigindo as datas de início das inscrições
for i in range(len(df)):
    if df.iloc[i]['registration_date_init'] is not np.nan and df.iloc[i]['registration_date_end'] is not np.nan:
        res = fixDateInit(df.iloc[i]['registration_date_init'], df.iloc[i]['registration_date_end'])
        if res:
            df.iloc[i]['registration_date_init'] = res
    

In [638]:
# Formatando as datas resultantes
df['registration_date_end'].replace('', np.nan, inplace=True, regex = True)
for i in range(len(df)):
    if not df.iloc[i]['registration_date_end']:
        df.iloc[i]['registration_date_end'] = np.nan
    if not df.iloc[i]['registration_date_end'] is np.nan:
        new_date = getDateFormated(df.iloc[i]['registration_date_end'])
        if new_date:
            df.iloc[i]['registration_date_end'] = new_date
            
for i in range(len(df)):
    if not df.iloc[i]['registration_date_init']:
        df.iloc[i]['registration_date_init'] = np.nan
    if not df.iloc[i]['registration_date_init'] is np.nan:
        new_date = getDateFormated(df.iloc[i]['registration_date_init'])
        if new_date:
            df.iloc[i]['registration_date_init'] = new_date

In [639]:
df[['registration_date_init','registration_date_end']]

,registration_date_init,registration_date_end
0,06/04/2021,03/05/2021
1,05/04/2020,27/04/2020
2,11/11/2019,06/12/2019
3,NaN,NaN
4,01/02/2021,12/02/2021
5,18/11/2019,17/12/2019
6,15/03/2021,25/03/2021
7,03/02/2021,09/03/2021
8,17/04/2019,17/05/2019
9,07/10/2020,13/10/2020


#### Tratando os valores monetários
O objetivo é transformar os valores monetários em números `decimal` no seguinte formato: `9999+.99`.

In [647]:
# Backup do campo value
df['value'].replace('', np.nan, inplace = True, regex = True)
df['value'].fillna(value = np.nan, inplace = True)
df_value = df[['value']]

In [651]:
# Convertendo str para decimal e tratando os valores
for i in range(len(df_value)):
    if df_value.iloc[i]['value'] is not np.nan:
        money = df_value.iloc[i]['value'].replace('.','')
        money = money.replace(',','.')
        new_value = Decimal(money)
        if new_value:
            df_value.iloc[i]['value'] = new_value

In [653]:
# Substituindo a tabela value pelos valores formatados adequadamente
df[['value']] = df_value[['value']]

In [642]:
# Total investido na cultura do ceará entre os anos coletados
df_value.sum()

value    463718106.77
dtype: object

In [643]:
df_value.head()

,value
0,3320000.00
1,1500000.00
2,6877492.00
3,3234.00
4,192000.00


### Gerando CSV
O objetivo é gerar um arquivo CSV contendo os dados coletados neste projeto. <br>
A estrutura do CSV (da esqurda para a direita do arquivo) sem a seguinte ordem de campos:

1. **title**: Título do edital
    - Formato dos dados: Caracteres maiúsculos (`[A-ZÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]`)
    
2. **value**: Valor total oferecido pelo edital
    - Formato dos dados: decimal com dois dígitos à direita do `.` (`[0-9]+\.([0-9]{2})`)
    
3. **registration_date_init**: Data inicial das inscrições 
    - Formato dos dados: Caracteres separados por `/` (`d{2}/\d{2}/\d{4}`)
    
4. **registration_date_end**: Data término das inscrições
    - Formato dos dados: Caracteres separados por `/` (`d{2}/\d{2}/\d{4}`)
5. **amount_date_extensions**: Quantidade de vezes que houve prorrogação das inscrições do edital
    - Formato dos dados: Números inteiros (`\d{1}`)
    
6. **categories**: Categorias (publico alvo) para o qual o edital está sendo direcionado
    - Formato dos dados: Caracteres maiúsculos separados por `;` (`[A-ZÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]+?;`)
    

In [657]:
# Selecionando a ordem das colunas
dataset_cultura_cearense = df[['title', 'value', 'registration_date_init', 'registration_date_end', 'amount_date_extensions', 'categories']]

In [659]:
# Gerando o arquivo CSV chamado dataset_cultura_cearense
file_name = 'dataset_cultura_cearense.csv'

dataset_cultura_cearense.to_csv(file_name, sep='\t', encoding='utf-8')

## Coletando os dados dos projetos aprovados pelos editais

#### Coletando dados dos projetos inscritos

In [762]:
# Lista de projetos inscritos por cada edital
registred_projects_list = [
    'editais/resultado_preliminar_habilitação_da_inscrição.pdf'
]

for r_project in registred_projects_list:
    df_registred_projects = tabula.read_pdf(r_project, pages = "all")
    amount_register = getAmoutRegisters(df_registred_projects)
    print(amount_register)

263


#### Coletando dados dos projetos aprovados

In [773]:
# Lista de projetos aprovados por cada edital 
approved_projects_list = [
    'editais/avaliaÇÃo_e_seleÇÃo_da_proposta_-_result_prelim._artes_visuais_(2)2.pdf'
]

for a_projects in approved_projects_list:
    df_approved_projects = tabula.read_pdf(a_projects, pages = "all")
    approved_projects_list = getApprovedProjects(df_approved_projects)
    print(approved_projects_list)

[['KELVIANE DE LIMA FERNANDES', 'RESIDÊNCIAS FORMATIVAS', 'FORTALEZA'], ['INSTITUTO UNIÃO DE ARTE EDUCAÇÃO E\rCULTURAS POPULARES', 'EXPOSIÇÃO FOLCLORE EM CENA', 'FORTALEZA'], ['GIULIANA LUIZA LIYE ODO', 'E ESTA É A TROMPA', 'FORTALEZA'], ['ANTONIO EDIVALDO FERREIRA', 'IV NORTE BIENAL DE ARTE', 'SOBRAL'], ['RAIMUNDO NONATO SOARES DA SILVA', 'MOSTRA DE ARTESANATO,\rEMPREENDEDORISMO E\rDESENVOLVIMENTO SUSTENTÁVEL', 'IPU'], ['PRISCILLA ALVES DE SOUSA', 'APRENDER A NADAR', 'FORTALEZA'], ['JOSÉ ALVES PIMENTA JÚNIOR', 'MAIS UM DIA: CLAUDIA SAMAPIO', 'FORTALEZA'], ['NÁDIA MARIA FERREIRA DE SOUSA', '/ARTE_SUBSTANTIVO FEMININO', 'FORTALEZA'], ['HÉRCULES GOMES DE LIMA', 'REVISTA DO LABORATÓRIO DE ARTE\rCONTEMPORÂNEA', 'FORTALEZA'], ['RAONI OLIVEIRA MARQUES', 'DICA HERÁLDICA', 'FORTALEZA'], ['RENATA GRACIANE FERREIRA DE SOUSA', 'CULTURA É A NOSSA PRAIA', 'FORTALEZA'], ['DANNIEL FERNANDES BEZERRA DE\rMENEZES', 'FILME ESPELHO DA VIDA', 'FORTALEZA'], ['FRANCISCO ADRIANO COSTA SOUZA', '“ALMA E MÃO”', 

In [775]:
approved_projects_list

[['KELVIANE DE LIMA FERNANDES', 'RESIDÊNCIAS FORMATIVAS', 'FORTALEZA'],
 ['INSTITUTO UNIÃO DE ARTE EDUCAÇÃO E\rCULTURAS POPULARES',
  'EXPOSIÇÃO FOLCLORE EM CENA',
  'FORTALEZA'],
 ['GIULIANA LUIZA LIYE ODO', 'E ESTA É A TROMPA', 'FORTALEZA'],
 ['ANTONIO EDIVALDO FERREIRA', 'IV NORTE BIENAL DE ARTE', 'SOBRAL'],
 ['RAIMUNDO NONATO SOARES DA SILVA',
  'MOSTRA DE ARTESANATO,\rEMPREENDEDORISMO E\rDESENVOLVIMENTO SUSTENTÁVEL',
  'IPU'],
 ['PRISCILLA ALVES DE SOUSA', 'APRENDER A NADAR', 'FORTALEZA'],
 ['JOSÉ ALVES PIMENTA JÚNIOR', 'MAIS UM DIA: CLAUDIA SAMAPIO', 'FORTALEZA'],
 ['NÁDIA MARIA FERREIRA DE SOUSA', '/ARTE_SUBSTANTIVO FEMININO', 'FORTALEZA'],
 ['HÉRCULES GOMES DE LIMA',
  'REVISTA DO LABORATÓRIO DE ARTE\rCONTEMPORÂNEA',
  'FORTALEZA'],
 ['RAONI OLIVEIRA MARQUES', 'DICA HERÁLDICA', 'FORTALEZA'],
 ['RENATA GRACIANE FERREIRA DE SOUSA', 'CULTURA É A NOSSA PRAIA', 'FORTALEZA'],
 ['DANNIEL FERNANDES BEZERRA DE\rMENEZES',
  'FILME ESPELHO DA VIDA',
  'FORTALEZA'],
 ['FRANCISCO ADRIANO CO

#### Gerando um novo dataset Cultura Cearense

In [776]:
new_dataset_cultura_cearense = dataset_cultura_cearense

In [792]:
# Criando o campo referente à quantidade de inscritos
new_dataset_cultura_cearense['amount_registers'] = np.nan

In [804]:
# Atribuindo o valor 
amount = getAmoutRegisters(df_registred_projects)
new_dataset_cultura_cearense.loc[2,'amount_registers'] = amount


In [805]:
# Verificando a quantidade de inscrições atribuído ao edital
new_dataset_cultura_cearense.loc[2,'amount_registers']

263.0

In [822]:
# Inserindo os projetos aprovados

# Criando o campo 'approved_projects'
new_dataset_cultura_cearense['approved_projects'] = np.nan

In [828]:
new_dataset_cultura_cearense.loc[2, 'approved_projects'] = approved_projects_list

In [843]:
# Fatia de projetos aprovados
approved_projects_list[0:3]

[['KELVIANE DE LIMA FERNANDES', 'RESIDÊNCIAS FORMATIVAS', 'FORTALEZA'],
 ['INSTITUTO UNIÃO DE ARTE EDUCAÇÃO E\rCULTURAS POPULARES',
  'EXPOSIÇÃO FOLCLORE EM CENA',
  'FORTALEZA'],
 ['GIULIANA LUIZA LIYE ODO', 'E ESTA É A TROMPA', 'FORTALEZA']]

In [832]:
list_ = new_dataset_cultura_cearense.values.tolist()

In [838]:
list_[2][7] = approved_projects_list

In [840]:
new_data_set = pd.DataFrame(list_)

In [842]:
# Gerando o arquivo CSV chamado dataset_cultura_cearense
file_name_2 = 'dataset_cultura_cearense_2.csv'

new_data_set.to_csv(file_name_2, sep='\t', encoding='utf-8')

## Configurações gerais 

In [107]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 0)